<a href="https://colab.research.google.com/github/ailab-nda/NLP/blob/main/7_1_6_audio_preprocessing_tutorial_jp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

「torchaudioによる音声データの取り扱い」
======
【原題】AUDIO MANIPULATION WITH TORCHAUDIO

【元URL】https://pytorch.org/tutorials/beginner/audio_preprocessing_tutorial.html

【翻訳】電通国際情報サービスISID AIトランスフォーメーションセンター　大串 和正

【日付】2021年4月17日

【チュトーリアル概要】<br>
``torchaudio``を使い、音声データの基本的な取り扱い方法を説明します。

英語版のチュートリアルサイトでは項目は分かれているがJupyter Notebookは同じである、

- 7.1 torchaudioによる音声データの取り扱い
- 7.2 音声データの入出力（AUDIO I/O）
- 7.3 データオーギュメンテーション（DATA AUGMENTATION）
- 7.4 特徴量抽出（FEATURE EXTRACTIONS）	
- 7.5 特徴量オーギュメンテーション（FEATURE AUGMENTATION）
- 7.6 データセット（DATASETS）

を本ページで解説します。

---


7.1 torchaudioによる音声データの取り扱い
==================================

``torchaudio`` には強力な音声データの入出力関数、前処理変換、データセットが実装されています。

本チュートリアルでは音声データの準備方法とニューラルネットワークモデルへ入力可能な特徴量の抽出方法を解説します。


In [ ]:
%matplotlib inline

In [ ]:
# Google Colabでこのチュートリアルを実行するには、
# 次のコマンドで必要なパッケージをインストールします
!pip install torchaudio librosa boto3

import torch
import torchaudio
import torchaudio.functional as F
import torchaudio.transforms as T

print(torch.__version__)
print(torchaudio.__version__)

データとユーティリティ関数の準備 (この節はセルを実行するだけです)
--------------------------------------------------------




In [ ]:
#@title データとユーティリティ関数の準備 {display-mode: "form"}
#@markdown
#@markdown このセルの中身を確認する必要はありません。
#@markdown
#@markdown **本セルを一度実行して、次のセルへと進んでください**
#@markdown
#@markdown このチュートリアルでは [VOiCES dataset](https://iqtlabs.github.io/voices/) のデータセットを使用します。ライセンスは Creative Commons BY 4.0 です。
#@markdown
#@markdown 　

#@markdown ※ **（日本語訳注）本セルをダブルクリックすると、非表示にしているプログラムが表示されます。**
#@markdown
#@markdown **本セルは「説明のテキスト・セル」ではなく、プログラムを非表示にしている「コードセル」なので、実行するのを忘れないように気をつけてください。** 
#@markdown
#@markdown Google Clabで本セルの右上のメニュ（「…」が縦）→「フォーム」→「コードを表示」or「コードを非表示」でコードをセルに表示するかどうか設定できます。
#@markdown

#-------------------------------------------------------------------------------
# Preparation of data and helper functions.
#-------------------------------------------------------------------------------
import io
import os
import math
import tarfile
import multiprocessing

import scipy
import librosa
import boto3
from botocore import UNSIGNED
from botocore.config import Config
import requests
import matplotlib
import matplotlib.pyplot as plt
from IPython.display import Audio, display

[width, height] = matplotlib.rcParams['figure.figsize']
if width < 10:
  matplotlib.rcParams['figure.figsize'] = [width * 2.5, height]

_SAMPLE_DIR = "_sample_data"
SAMPLE_WAV_URL = "https://pytorch-tutorial-assets.s3.amazonaws.com/steam-train-whistle-daniel_simon.wav"
SAMPLE_WAV_PATH = os.path.join(_SAMPLE_DIR, "steam.wav")

SAMPLE_WAV_SPEECH_URL = "https://pytorch-tutorial-assets.s3.amazonaws.com/VOiCES_devkit/source-16k/train/sp0307/Lab41-SRI-VOiCES-src-sp0307-ch127535-sg0042.wav"
SAMPLE_WAV_SPEECH_PATH = os.path.join(_SAMPLE_DIR, "speech.wav")

SAMPLE_RIR_URL = "https://pytorch-tutorial-assets.s3.amazonaws.com/VOiCES_devkit/distant-16k/room-response/rm1/impulse/Lab41-SRI-VOiCES-rm1-impulse-mc01-stu-clo.wav"
SAMPLE_RIR_PATH = os.path.join(_SAMPLE_DIR, "rir.wav")

SAMPLE_NOISE_URL = "https://pytorch-tutorial-assets.s3.amazonaws.com/VOiCES_devkit/distant-16k/distractors/rm1/babb/Lab41-SRI-VOiCES-rm1-babb-mc01-stu-clo.wav"
SAMPLE_NOISE_PATH = os.path.join(_SAMPLE_DIR, "bg.wav")

SAMPLE_MP3_URL = "https://pytorch-tutorial-assets.s3.amazonaws.com/steam-train-whistle-daniel_simon.mp3"
SAMPLE_MP3_PATH = os.path.join(_SAMPLE_DIR, "steam.mp3")

SAMPLE_GSM_URL = "https://pytorch-tutorial-assets.s3.amazonaws.com/steam-train-whistle-daniel_simon.gsm"
SAMPLE_GSM_PATH = os.path.join(_SAMPLE_DIR, "steam.gsm")

SAMPLE_TAR_URL = "https://pytorch-tutorial-assets.s3.amazonaws.com/VOiCES_devkit.tar.gz"
SAMPLE_TAR_PATH = os.path.join(_SAMPLE_DIR, "sample.tar.gz")
SAMPLE_TAR_ITEM = "VOiCES_devkit/source-16k/train/sp0307/Lab41-SRI-VOiCES-src-sp0307-ch127535-sg0042.wav"

S3_BUCKET = "pytorch-tutorial-assets"
S3_KEY = "VOiCES_devkit/source-16k/train/sp0307/Lab41-SRI-VOiCES-src-sp0307-ch127535-sg0042.wav"

YESNO_DATASET_PATH = os.path.join(_SAMPLE_DIR, "yes_no")
os.makedirs(YESNO_DATASET_PATH, exist_ok=True)
os.makedirs(_SAMPLE_DIR, exist_ok=True)

def _fetch_data():
  uri = [
    (SAMPLE_WAV_URL, SAMPLE_WAV_PATH),
    (SAMPLE_WAV_SPEECH_URL, SAMPLE_WAV_SPEECH_PATH),
    (SAMPLE_RIR_URL, SAMPLE_RIR_PATH),
    (SAMPLE_NOISE_URL, SAMPLE_NOISE_PATH),
    (SAMPLE_MP3_URL, SAMPLE_MP3_PATH),
    (SAMPLE_GSM_URL, SAMPLE_GSM_PATH),
    (SAMPLE_TAR_URL, SAMPLE_TAR_PATH),
  ]
  for url, path in uri:
    with open(path, 'wb') as file_:
      file_.write(requests.get(url).content)

_fetch_data()

def _download_yesno():
  if os.path.exists(os.path.join(YESNO_DATASET_PATH, "waves_yesno.tar.gz")):
    return
  torchaudio.datasets.YESNO(root=YESNO_DATASET_PATH, download=True)

YESNO_DOWNLOAD_PROCESS = multiprocessing.Process(target=_download_yesno)
YESNO_DOWNLOAD_PROCESS.start()

def _get_sample(path, resample=None):
  effects = [
    ["remix", "1"]
  ]
  if resample:
    effects.append(["rate", f'{resample}'])
  return torchaudio.sox_effects.apply_effects_file(path, effects=effects)

def get_speech_sample(*, resample=None):
  return _get_sample(SAMPLE_WAV_SPEECH_PATH, resample=resample)

def get_sample(*, resample=None):
  return _get_sample(SAMPLE_WAV_PATH, resample=resample)

def get_rir_sample(*, resample=None, processed=False):
  rir_raw, sample_rate = _get_sample(SAMPLE_RIR_PATH, resample=resample)
  if not processed:
    return rir_raw, sample_rate
  rir = rir_raw[:, int(sample_rate*1.01):int(sample_rate*1.3)]
  rir = rir / torch.norm(rir, p=2)
  rir = torch.flip(rir, [1])
  return rir, sample_rate

def get_noise_sample(*, resample=None):
  return _get_sample(SAMPLE_NOISE_PATH, resample=resample)

def print_metadata(metadata, src=None):
  if src:
    print("-" * 10)
    print("Source:", src)
    print("-" * 10)
  print(" - sample_rate:", metadata.sample_rate)
  print(" - num_channels:", metadata.num_channels)
  print(" - num_frames:", metadata.num_frames)
  print(" - bits_per_sample:", metadata.bits_per_sample)
  print(" - encoding:", metadata.encoding)
  print()

def print_stats(waveform, sample_rate=None, src=None):
  if src:
    print("-" * 10)
    print("Source:", src)
    print("-" * 10)
  if sample_rate:
    print("Sample Rate:", sample_rate)
  print("Shape:", tuple(waveform.shape))
  print("Dtype:", waveform.dtype)
  print(f" - Max:     {waveform.max().item():6.3f}")
  print(f" - Min:     {waveform.min().item():6.3f}")
  print(f" - Mean:    {waveform.mean().item():6.3f}")
  print(f" - Std Dev: {waveform.std().item():6.3f}")
  print()
  print(waveform)
  print()

def plot_waveform(waveform, sample_rate, title="Waveform", xlim=None, ylim=None):
  waveform = waveform.numpy()

  num_channels, num_frames = waveform.shape
  time_axis = torch.arange(0, num_frames) / sample_rate

  figure, axes = plt.subplots(num_channels, 1)
  if num_channels == 1:
    axes = [axes]
  for c in range(num_channels):
    axes[c].plot(time_axis, waveform[c], linewidth=1)
    axes[c].grid(True)
    if num_channels > 1:
      axes[c].set_ylabel(f'Channel {c+1}')
    if xlim:
      axes[c].set_xlim(xlim)
    if ylim:
      axes[c].set_ylim(ylim)
  figure.suptitle(title)
  plt.show(block=False)

def plot_specgram(waveform, sample_rate, title="Spectrogram", xlim=None):
  waveform = waveform.numpy()

  num_channels, num_frames = waveform.shape
  time_axis = torch.arange(0, num_frames) / sample_rate

  figure, axes = plt.subplots(num_channels, 1)
  if num_channels == 1:
    axes = [axes]
  for c in range(num_channels):
    axes[c].specgram(waveform[c], Fs=sample_rate)
    if num_channels > 1:
      axes[c].set_ylabel(f'Channel {c+1}')
    if xlim:
      axes[c].set_xlim(xlim)
  figure.suptitle(title)
  plt.show(block=False)

def play_audio(waveform, sample_rate):
  waveform = waveform.numpy()

  num_channels, num_frames = waveform.shape
  if num_channels == 1:
    display(Audio(waveform[0], rate=sample_rate))
  elif num_channels == 2:
    display(Audio((waveform[0], waveform[1]), rate=sample_rate))
  else:
    raise ValueError("Waveform with more than 2 channels are not supported.")

def inspect_file(path):
  print("-" * 10)
  print("Source:", path)
  print("-" * 10)
  print(f" - File size: {os.path.getsize(path)} bytes")
  print_metadata(torchaudio.info(path))

def plot_spectrogram(spec, title=None, ylabel='freq_bin', aspect='auto', xmax=None):
  fig, axs = plt.subplots(1, 1)
  axs.set_title(title or 'Spectrogram (db)')
  axs.set_ylabel(ylabel)
  axs.set_xlabel('frame')
  im = axs.imshow(librosa.power_to_db(spec), origin='lower', aspect=aspect)
  if xmax:
    axs.set_xlim((0, xmax))
  fig.colorbar(im, ax=axs)
  plt.show(block=False)

def plot_mel_fbank(fbank, title=None):
  fig, axs = plt.subplots(1, 1)
  axs.set_title(title or 'Filter bank')
  axs.imshow(fbank, aspect='auto')
  axs.set_ylabel('frequency bin')
  axs.set_xlabel('mel bin')
  plt.show(block=False)

def get_spectrogram(
    n_fft = 400,
    win_len = None,
    hop_len = None,
    power = 2.0,
):
  waveform, _ = get_speech_sample()
  spectrogram = T.Spectrogram(
      n_fft=n_fft,
      win_length=win_len,
      hop_length=hop_len,
      center=True,
      pad_mode="reflect",
      power=power,
  )
  return spectrogram(waveform)

def plot_pitch(waveform, sample_rate, pitch):
  figure, axis = plt.subplots(1, 1)
  axis.set_title("Pitch Feature")
  axis.grid(True)

  end_time = waveform.shape[1] / sample_rate
  time_axis = torch.linspace(0, end_time,  waveform.shape[1])
  axis.plot(time_axis, waveform[0], linewidth=1, color='gray', alpha=0.3)

  axis2 = axis.twinx()
  time_axis = torch.linspace(0, end_time, pitch.shape[1])
  ln2 = axis2.plot(
      time_axis, pitch[0], linewidth=2, label='Pitch', color='green')

  axis2.legend(loc=0)
  plt.show(block=False)

def plot_kaldi_pitch(waveform, sample_rate, pitch, nfcc):
  figure, axis = plt.subplots(1, 1)
  axis.set_title("Kaldi Pitch Feature")
  axis.grid(True)

  end_time = waveform.shape[1] / sample_rate
  time_axis = torch.linspace(0, end_time,  waveform.shape[1])
  axis.plot(time_axis, waveform[0], linewidth=1, color='gray', alpha=0.3)

  time_axis = torch.linspace(0, end_time, pitch.shape[1])
  ln1 = axis.plot(time_axis, pitch[0], linewidth=2, label='Pitch', color='green')
  axis.set_ylim((-1.3, 1.3))

  axis2 = axis.twinx()
  time_axis = torch.linspace(0, end_time, nfcc.shape[1])
  ln2 = axis2.plot(
      time_axis, nfcc[0], linewidth=2, label='NFCC', color='blue', linestyle='--')

  lns = ln1 + ln2
  labels = [l.get_label() for l in lns]
  axis.legend(lns, labels, loc=0)
  plt.show(block=False)

7.2 音声データの入出力（AUDIO I/O）
=========

torchaudio には ``libsox`` が統合されており、様々な音声データの入出力が可能です。




音声データのメタデータ取得
----------------------

``torchaudio.info``関数を用いて音声データのメタデータを取得できます。

この関数にはファイルパス（のオブジェクト）かファイル（のオブジェクト）を渡すことができます。




In [ ]:
metadata = torchaudio.info(SAMPLE_WAV_PATH)
print_metadata(metadata, src=SAMPLE_WAV_PATH)

# 日本語訳注
# 上記で使用しているSAMPLE_WAV_PATHは、
# セル「データとユーティリティ関数の準備
# このセルの中身を確認する必要はありません。
# 本セルを一度実行して、次のセルへと進んでください」を
# 実行した際に作られる変数です

各出力の意味は以下の通りです。

-  ``sample_rate`` 音声データのサンプリングレート
-  ``num_channels`` チャネル数
-  ``num_frames`` チャネル毎のフレーム数
-  ``bits_per_sample`` ビット深度
-  ``encoding`` エンコードのフォーマット

``encoding`` は以下のいずれかの値をとります。

-  ``"PCM_S"``: 符号付き整数型リニア PCM
-  ``"PCM_U"``: 符号なし整数型リニア PCM
-  ``"PCM_F"``: 浮動小数点数型リニア PCM
-  ``"FLAC"``: Flac, [Free Lossless Audio
   Codec](https://xiph.org/flac/)
-  ``"ULAW"``: Mu-law,
   [\[wikipedia\]](https://en.wikipedia.org/wiki/%CE%9C-law_algorithm)
-  ``"ALAW"``: A-law
   [\[wikipedia\]](https://en.wikipedia.org/wiki/A-law_algorithm)
-  ``"MP3"`` : MP3, MPEG-1 Audio Layer III
-  ``"VORBIS"``: OGG Vorbis [\[xiph.org\]](https://xiph.org/vorbis/)
-  ``"AMR_NB"``: Adaptive Multi-Rate
   [\[wikipedia\]](https://en.wikipedia.org/wiki/Adaptive_Multi-Rate_audio_codec)
-  ``"AMR_WB"``: Adaptive Multi-Rate Wideband
   [\[wikipedia\]](https://en.wikipedia.org/wiki/Adaptive_Multi-Rate_Wideband)
-  ``"OPUS"``: Opus [\[opus-codec.org\]](https://opus-codec.org/)
-  ``"GSM"``: GSM-FR
   [\[wikipedia\]](https://en.wikipedia.org/wiki/Full_Rate)
-  ``"UNKNOWN"`` 上記のいずれにも該当しないフォーマット




**注意**

- データが圧縮されていたり可変ビットレートであったりする場合 (mp3 など) は ``bits_per_sample`` は ``0`` となることがあります。
-  GSM-FRフォーマットでは ``num_frames`` が ``0`` になることがあります。




In [ ]:
metadata = torchaudio.info(SAMPLE_MP3_PATH)
print_metadata(metadata, src=SAMPLE_MP3_PATH)

metadata = torchaudio.info(SAMPLE_GSM_PATH)
print_metadata(metadata, src=SAMPLE_GSM_PATH)

### ファイル・オブジェクトの情報取得

``info``関数は ファイルオブジェクト（file-like オブジェクト）に対しても使用可能です。




In [ ]:
with requests.get(SAMPLE_WAV_URL, stream=True) as response:
  metadata = torchaudio.info(response.raw)
print_metadata(metadata, src=SAMPLE_WAV_URL)

**注意** 

file-like オブジェクトを渡すと ``info`` 関数はデータ全体ではなく、データの最初の領域だけを読み取ります。

従って、音声データのフォーマットによっては、フォーマット自体の情報を含む、メタデータを正しく読み取れないことがあります。以下はその例です。

-  ``format`` 引数を使って音声データのフォーマットを明示する。
-  返されるメタデータでは ``num_frames = 0`` となっている。




In [ ]:
with requests.get(SAMPLE_MP3_URL, stream=True) as response:
  metadata = torchaudio.info(response.raw, format="mp3")

  print(f"Fetched {response.raw.tell()} bytes.")
print_metadata(metadata, src=SAMPLE_MP3_URL)


# 日本語訳注
# SAMPLE_MP3_URLをファイルパスのオブジェクトとしてinfoに渡した3つほど上のセルでは
# きちんと情報が取得できていますが、パスからファイルを取ってきてファイルのオブジェクトで
# 渡した本セルでは、num_framesなどが0になりきちんと、読み取れていません。

音声データを読み込みテンソルにする
------------------------------

音声データの読み込みには ``torchaudio.load`` 関数が使えます。

この関数は path-like オブジェクトと file-like オブジェクトを受け取ることができます。

<br>

戻り値は波形データ (``Tensor``) のタプルとサンプルレート (``int``) です。

デフォルトでは、返されるテンソルオブジェクトの値は ``dtype=torch.float32`` であり、範囲は ``[-1.0, 1.0]`` に正規化されます。


<br>

サポートされているフォーマットの一覧は [torchaudio のドキュメント](https://pytorch.org/audio)を参照してください。

In [ ]:
waveform, sample_rate = torchaudio.load(SAMPLE_WAV_SPEECH_PATH)

print_stats(waveform, sample_rate=sample_rate)
plot_waveform(waveform, sample_rate)
plot_specgram(waveform, sample_rate)
play_audio(waveform, sample_rate)

# 日本語訳注
# 上記で使用しているplot_waveformなどの描画関数、
# play_audioという音声再生関数は
#
# セル「データとユーティリティ関数の準備
# このセルの中身を確認する必要はありません。
# 本セルを一度実行して、次のセルへと進んでください」
#
#を実行した際に定義されています

**日本語訳注**

この音声データは今後も使うのですが、おそらく

I had the curiosity beside me this moment.

もしくは

I had that curiosity beside me at this moment.（Amazon Transcribeで文字にした場合）

と言っており、

<br>

「私はこのときから、好奇心を抱くようになりました。」

もしくは

「私はこの瞬間、好奇心を持っていました。」という意味でしょうか。。

---


### file-like オブジェクトの読み込み

``torchaudio``の入出力関数は file-like オブジェクトをサポートしています。

そのためローカル以外の場所から、音声データ取得とデコードを同時に行うことが可能です。

次のセルで例を示します。



In [ ]:
# HTTPリクエストとして音声データを読み込む
with requests.get(SAMPLE_WAV_SPEECH_URL, stream=True) as response:
  waveform, sample_rate = torchaudio.load(response.raw)
plot_specgram(waveform, sample_rate, title="HTTP datasource")

# tarファイルから音声データを読み込む
with tarfile.open(SAMPLE_TAR_PATH, mode='r') as tarfile_:
  fileobj = tarfile_.extractfile(SAMPLE_TAR_ITEM)
  waveform, sample_rate = torchaudio.load(fileobj)
plot_specgram(waveform, sample_rate, title="TAR file")

# S3から音声データを読み込む
client = boto3.client('s3', config=Config(signature_version=UNSIGNED))
response = client.get_object(Bucket=S3_BUCKET, Key=S3_KEY)
waveform, sample_rate = torchaudio.load(response['Body'])
plot_specgram(waveform, sample_rate, title="From S3")

### スライシングのTips

``num_frames`` と ``frame_offset`` を引数に与えるとデコード時にスライスされたテンソルが返されます。

通常のテンソルのスライシング (``waveform[:, frame_offset:frame_offset+num_frames]``) でも同じ結果は得られますが、``num_frames`` と ``frame_offset`` 引数を使う方が効率的です。

<br>

なぜならデコード時に、 ``torchaudio.load`` 関数が指定されたフレームが完了した段階でデータの取得とデコードを終えるからです。

必要なデータが取得された時点でデータ転送が停止するため、ネットワークを通じて音声データを転送する場合にこの仕様は有用です。


次の例で説明します。


In [ ]:
# 異なる2種類のデコード手法を説明します。
# 1つ目はデータ全体を取得し、それからデコードする方法です。
# 2つ目は必要なデコードが完了した時点でデータの取得を終了する方法です。
# 結果として得られる波形は同一のものとなります。
# ですが、取得したデータ量は2つ目の方法が少なく済みます。

frame_offset, num_frames = 16000, 16000  # 1秒から2秒のデータを取得しデコードする

# [1]
print("データ全体を取得...")
with requests.get(SAMPLE_WAV_SPEECH_URL, stream=True) as response:
  waveform1, sample_rate1 = torchaudio.load(response.raw)
  waveform1 = waveform1[:, frame_offset:frame_offset+num_frames]
  print(f" - {response.raw.tell()} バイトのデータを取得しました。")

# [2]
print("要求されたフレームが利用可能になるまでデータを取得...")
with requests.get(SAMPLE_WAV_SPEECH_URL, stream=True) as response:
  waveform2, sample_rate2 = torchaudio.load(
      response.raw, frame_offset=frame_offset, num_frames=num_frames)
  print(f" - {response.raw.tell()} バイトのデータを取得しました。")

print("得られた波形の確認 ... ", end="")
assert (waveform1 == waveform2).all()
print("同一の波形です")

音声データをファイルへ保存する
--------------------

``torchaudio.save`` 関数を使い、一般的なアプリケーションで取り扱える
フォーマットで音声データを保存することができます。

この関数はpath-likeオブジェクトとfile-likeオブジェクトを受け取ることができます。

<br>

file-likeオブジェクトを渡す場合は、フォーマットを明示するために ``format`` 引数が必要です。

path-like オブジェクトの場合は拡張子からフォーマットが決定されます。拡張子なしで保存する場合は``format`` 引数が必要です。

<br>

WAVフォーマット保存時、``float32``型のテンソルに対するデフォルトのエンコードは32ビット浮動小数点数CPMです。``encoding``と``bits_per_sample``引数によって変更可能です。

例えば16ビット符号付き整数PCMで保存をするには以下のセルのように実行します。

<br>

**注意** 

少ないビット深度でデータを保存するとファイルサイズは削減されますが精度が低下します。


In [ ]:
waveform, sample_rate = get_sample()
print_stats(waveform, sample_rate=sample_rate)

# エンコードを指定せずに保存する
# データに適合したエンコードが自動的に選択される
path = "save_example_default.wav"
torchaudio.save(path, waveform, sample_rate)
inspect_file(path)

# 16ビット符号付き整数リニアPCMとして保存する
# ファイルサイズは半分になりますが、精度が低下する
path = "save_example_PCM_S16.wav"
torchaudio.save(
    path, waveform, sample_rate,
    encoding="PCM_S", bits_per_sample=16)
inspect_file(path)

``torchaudio.save`` は他のフォーマットにも対応しています。いくつか例を挙げます。




In [ ]:
waveform, sample_rate = get_sample(resample=8000)

formats = [
  "mp3",
  "flac",
  "vorbis",
  "sph",
  "amb",
  "amr-nb",
  "gsm",
]

for format in formats:
  path = f"save_example.{format}"
  torchaudio.save(path, waveform, sample_rate, format=format)
  inspect_file(path)

### file-likeオブジェクトへ保存する

音声データをfile-likeオブジェクトに保存することができます。


file-likeオブジェクトに保存する場合、``format``引数が必須となります。

In [ ]:
waveform, sample_rate = get_sample()

# バイトバッファへ保存する
buffer_ = io.BytesIO()
torchaudio.save(buffer_, waveform, sample_rate, format="wav")

buffer_.seek(0)
print(buffer_.read(16))

7.3 データオーギュメンテーション（DATA AUGMENTATION）
=================

``torchaudio`` には様々なデータオーギュメンテーション手法が用意されています。




エフェクトとフィルタを適用する
------------------------------

``torchaudio.sox_effects``モジュールを使うと音声データのテンソルオブジェクトやファイルオブジェクトに対し、``sox``コマンドのように直接フィルタを適用できます。

<br>

フィルタの適用には2つの関数があります。

-  ``torchaudio.sox_effects.apply_effects_tensor`` はテンソルにエフェクトを適用します
-  ``torchaudio.sox_effects.apply_effects_file`` はその他の音声データソースにエフェクトを適用します

<br>

どちらの関数もエフェクトを``List[List[str]]``の形式で指定できます。

これは``sox``コマンドの動作にほぼ一致していますが、1つ注意が必要なのは
``sox``コマンドは自動的にいくつかのエフェクトが追加されるのに対して、
torchaudioの実装はそうなっていない点です。

利用可能なエフェクトの一覧は[soxのドキュメント](http://sox.sourceforge.net/sox.html)を御覧ください。






**Tip** 

音声データをリアルタイムに読み込んでリサンプルする必要がある場合、
``torchaudio.sox_effects.apply_effects_file``で``"rate"``を使用してください。

**注意** 

``apply_effects_file`` 関数はfile-likeオブジェクトもpath-likeオブジェクトも受け取ることができます。

``torchaudio.load``と同じく、ファイルの拡張子かヘッダーから音声フォーマットが読み取れない場合、``format``引数を使い関数に音声データのフォーマットを知らせることができます。

**注意** 

この操作は微分不可能です。

In [ ]:
# データを読み込む
waveform1, sample_rate1 = get_sample(resample=16000)

# エフェクトを定義する
effects = [
  ["lowpass", "-1", "300"], # ローパスフィルター(単極)の適用
  ["speed", "0.8"],  # 速度を下げる
                     # サンプリングレートだけが変更されるので、 
                     # この後に元のサンプリングレートの値で
                     # `rate`エフェクトを適用する必要がある
  ["rate", f"{sample_rate1}"],
  ["reverb", "-w"],  # 反響のエフェクトを加えるとドラマチックな雰囲気になる
]

# エフェクトを適用する
waveform2, sample_rate2 = torchaudio.sox_effects.apply_effects_tensor(
    waveform1, sample_rate1, effects)

plot_waveform(waveform1, sample_rate1, title="Original", xlim=(-.1, 3.2))
plot_waveform(waveform2, sample_rate2, title="Effects Applied", xlim=(-.1, 3.2))
print_stats(waveform1, sample_rate=sample_rate1, src="Original")
print_stats(waveform2, sample_rate=sample_rate2, src="Effects Applied")

エフェクト適用後にフレーム数とチャンネル数が元のデータから変わっていることに注意してください。

実際に音声を聞いてみましょう。

データオーギュメンテーションの操作により音声が変化しており、後者の方がよりドラマチックに感じませんか。


In [ ]:
plot_specgram(waveform1, sample_rate1, title="Original", xlim=(0, 3.04))
play_audio(waveform1, sample_rate1)
plot_specgram(waveform2, sample_rate2, title="Effects Applied", xlim=(0, 3.04))
play_audio(waveform2, sample_rate2)

室内での反響をシミュレーションする
----------------------------

[Convolution reverb](https://en.wikipedia.org/wiki/Convolution_reverb) は異なる環境で音が鳴っているかのような、クリーンな音声データを生成する際に使われる技術です。

<br>

Room Impluse Response (RIR, 室内インパルス応答)により、会議室で発せられたようなクリーンな音声データを生成できます。

<br>

この処理を行うためにはRIRデータが必要です。

次のセルではVOiCESデータセットの音声を使用していますが、ご自身で録音した音声データを使うこともできます。

マイクを有効にし、ご自身で手を叩いて拍手の音を収録してみてください。

<br>

日本語訳注：

ご自身が扱いたい、特定の部屋や環境での反響をシミュレーションする際には、その反響のデータとして、一度拍手をした音声データ（インパルス応答のデータ）を用意します。


In [ ]:
sample_rate = 8000

rir_raw, _ = get_rir_sample(resample=sample_rate)

plot_waveform(rir_raw, sample_rate, title="Room Impulse Response (raw)", ylim=None)
plot_specgram(rir_raw, sample_rate, title="Room Impulse Response (raw)")
play_audio(rir_raw, sample_rate)

最初にRIRを整えます。主要インパルスを抽出し、振幅を正規化し、時間軸を反転します。


In [ ]:
rir = rir_raw[:, int(sample_rate*1.01):int(sample_rate*1.3)]
rir = rir / torch.norm(rir, p=2)
rir = torch.flip(rir, [1])

print_stats(rir)
plot_waveform(rir, sample_rate, title="Room Impulse Response", ylim=None)

次にスピーチの信号とRIRフィルターの畳み込みを行います。

In [ ]:
speech, _ = get_speech_sample(resample=sample_rate)

speech_ = torch.nn.functional.pad(speech, (rir.shape[1]-1, 0))
augmented = torch.nn.functional.conv1d(speech_[None, ...], rir[None, ...])[0]

plot_waveform(speech, sample_rate, title="Original", ylim=None)
plot_waveform(augmented, sample_rate, title="RIR Applied", ylim=None)

plot_specgram(speech, sample_rate, title="Original")
play_audio(speech, sample_rate)

plot_specgram(augmented, sample_rate, title="RIR Applied")
play_audio(augmented, sample_rate)

# 日本語訳注
# 再生ファイルが以下に2つ作成されます。
# 1つ目はOriginalのクリーンなファイルです。
# 2つ目は拍手で取得した室内反響をOriginalなデータに畳み込み、室内で収録したような音声にしたファイルです。
# 2つのデータを再生し、違いを感じてみてください。

バックグラウンドノイズの付加
-----------------------

音声データにバックグラウンドノイズを付加するには、単に音声データのテンソルと
ノイズテンソルを足し合わせます。

<be>

ノイズの強度を調整する一般的な手法には[Signal-to-Noise Ratio (SNR、信号雑音比)](https://ja.wikipedia.org/wiki/SN%E6%AF%94#:~:text=SN%E6%AF%94%EF%BC%88%E3%82%A8%E3%82%B9%E3%82%A8%E3%83%8C%E3%81%B2%EF%BC%89%E3%81%AF,%E3%80%81S%2FN%E3%81%A8%E3%82%82%E7%95%A5%E3%81%99%E3%80%82)があります。

\begin{align}\mathrm{SNR} = \frac{P_\mathrm{signal}}{P_\mathrm{noise}}\end{align}

\begin{align}{\mathrm  {SNR_{{dB}}}}=10\log _{{10}}\left({\mathrm  {SNR}}\right)\end{align}




In [ ]:
sample_rate = 8000
speech, _ = get_speech_sample(resample=sample_rate)
noise, _ = get_noise_sample(resample=sample_rate)
noise = noise[:, :speech.shape[1]]

plot_waveform(noise, sample_rate, title="Background noise")
plot_specgram(noise, sample_rate, title="Background noise")
play_audio(noise, sample_rate)

speech_power = speech.norm(p=2)
noise_power = noise.norm(p=2)

for snr_db in [20, 10, 3]:
  snr = math.exp(snr_db / 10)
  scale = snr * noise_power / speech_power
  noisy_speech = (scale * speech + noise) / 2

  plot_waveform(noisy_speech, sample_rate, title=f"SNR: {snr_db} [dB]")
  plot_specgram(noisy_speech, sample_rate, title=f"SNR: {snr_db} [dB]")
  play_audio(noisy_speech, sample_rate)


# 日本語訳注
# 出力結果に4つの音声再生が用意されます。
# 1つ目はバックグラウンドノイズ音声の再生です。
# snr_dbが20,10,3のため、
# 2つ目から4つ目にかけて、音声再生の上から順にノイズが強い音声になっていることを確認してみてください。

テンソルオブジェクトにコーデックを適用する
-------------------------------

``torchaudio.functional.apply_codec``を使ってテンソルオブジェクトにコーデックを適用することができます。

**注意** この処理は微分不可能です。




In [ ]:
waveform, sample_rate = get_speech_sample(resample=8000)

plot_specgram(waveform, sample_rate, title="Original")
play_audio(waveform, sample_rate)

configs = [
    ({"format": "wav", "encoding": 'ULAW', "bits_per_sample": 8}, "8 bit mu-law"),
    ({"format": "gsm"}, "GSM-FR"),
    ({"format": "mp3", "compression": -9}, "MP3"),
    ({"format": "vorbis", "compression": -1}, "Vorbis"),
]
for param, title in configs:
  augmented = F.apply_codec(waveform, sample_rate, **param)
  plot_specgram(augmented, sample_rate, title=title)
  play_audio(augmented, sample_rate)

電話音声のシミュレーション
---------------------------

これまでの操作を組み合わせて、反響する部屋で、バックグラウンドで人々が話している中、電話を使って話している人の音声をシミュレーションできます。



In [ ]:
sample_rate = 16000
speech, _ = get_speech_sample(resample=sample_rate)

plot_specgram(speech, sample_rate, title="Original")
play_audio(speech, sample_rate)

# RIRを適用する
rir, _ = get_rir_sample(resample=sample_rate, processed=True)
speech_ = torch.nn.functional.pad(speech, (rir.shape[1]-1, 0))
speech = torch.nn.functional.conv1d(speech_[None, ...], rir[None, ...])[0]

plot_specgram(speech, sample_rate, title="RIR Applied")
play_audio(speech, sample_rate)

# バックグラウンドノイズを付加する
# ノイズは実環境において録音されたものなので、ノイズはその環境における音声特性を
# 既に持っていると考えます。従って、ノイズはRIRを適用した後に付加します。
noise, _ = get_noise_sample(resample=sample_rate)
noise = noise[:, :speech.shape[1]]

snr_db = 8
scale = math.exp(snr_db / 10) * noise.norm(p=2) / speech.norm(p=2)
speech = (scale * speech + noise) / 2

plot_specgram(speech, sample_rate, title="BG noise added")
play_audio(speech, sample_rate)

# フィルタの適用し、サンプリングレートを変更する
speech, sample_rate = torchaudio.sox_effects.apply_effects_tensor(
  speech,
  sample_rate,
  effects=[
      ["lowpass", "4000"],
      ["compand", "0.02,0.05", "-60,-60,-30,-10,-20,-8,-5,-8,-2,-8", "-8", "-7", "0.05"],
      ["rate", "8000"],
  ],
)

plot_specgram(speech, sample_rate, title="Filtered")
play_audio(speech, sample_rate)

# 電話用コーデックを適用する
speech = F.apply_codec(speech, sample_rate, format="gsm")

plot_specgram(speech, sample_rate, title="GSM Codec Applied")
play_audio(speech, sample_rate)

7.4 特徴量抽出（FEATURE EXTRACTIONS）
===================

``torchaudio``には音声データの領域で広く使われている特徴量抽出手法が
用意されています。

それらは``torchaudio.functional``と``torchaudio.transforms``
で利用可能です。




``functional``モジュールは単一の関数として機能を実装しています。
これらの関数はステートレスです。

<br>

``transforms``モジュールは、``functional``と``torch.nn.Module``の実装を用いて、オブジェクト指向で機能を実装しています。

すべての``transforms``は``torch.nn.Module``のサブクラスなので、TorchScriptによって繋げることが可能です。


利用可能な機能の一覧は[ドキュメント](https://pytorch.org/audio/stable/transforms.html)を参照してください。

<br>


本チュートリアルでは、時間領域と周波数領域間の変換 (``Spectrogram``, ``GriffinLim``,``MelSpectrogram``)とSpecAugomentと呼ばれるオーギュメンテーション手法 について説明します。

スペクトログラム
-----------

``Spectrogram``変換により、音声信号の周波数表現が得られます。




In [ ]:
waveform, sample_rate = get_speech_sample()

n_fft = 1024
win_length = None
hop_length = 512

# 変換の定義
spectrogram = T.Spectrogram(
    n_fft=n_fft,
    win_length=win_length,
    hop_length=hop_length,
    center=True,
    pad_mode="reflect",
    power=2.0,
)
# 変換の実行
spec = spectrogram(waveform)

print_stats(spec)
plot_spectrogram(spec[0], title='torchaudio')

GriffinLim
----------

``GriffinLim``により、スペクトログラムから波形データを復元できます。




In [ ]:
torch.random.manual_seed(0)
waveform, sample_rate = get_speech_sample()
plot_waveform(waveform, sample_rate, title="Original")
play_audio(waveform, sample_rate)

n_fft = 1024
win_length = None
hop_length = 512

spec = T.Spectrogram(
    n_fft=n_fft,
    win_length=win_length,
    hop_length=hop_length,
)(waveform)

griffin_lim = T.GriffinLim(
    n_fft=n_fft,
    win_length=win_length,
    hop_length=hop_length,
)
waveform = griffin_lim(spec)

plot_waveform(waveform, sample_rate, title="Reconstructed")
play_audio(waveform, sample_rate)

メルフィルタバンク
---------------

``torchaudio.functional.create_fb_matrix``は周波数ビンをメルスケールビンに変換するためのフィルタバンクを生成します。

この関数は入力の音声データ、特徴量を必要としないため``torchaudio.transforms``には対応した関数がありません。


In [ ]:
n_fft = 256
n_mels = 64
sample_rate = 6000

mel_filters = F.create_fb_matrix(
    int(n_fft // 2 + 1),
    n_mels=n_mels,
    f_min=0.,
    f_max=sample_rate/2.,
    sample_rate=sample_rate,
    norm='slaney'
)
plot_mel_fbank(mel_filters, "Mel Filter Bank - torchaudio")

### librosaとの比較

比較として、``librosa``で同様のメルフィルタバンクを生成します。

**注意** 

現状では、``htk=True``時のみ結果は一致します。

``htk=False``時の結果一致は``torchaudio``はサポートしていません。


In [ ]:
mel_filters_librosa = librosa.filters.mel(
    sample_rate,
    n_fft,
    n_mels=n_mels,
    fmin=0.,
    fmax=sample_rate/2.,
    norm='slaney',
    htk=True,
).T

plot_mel_fbank(mel_filters_librosa, "Mel Filter Bank - librosa")

mse = torch.square(mel_filters - mel_filters_librosa).mean().item()
print('Mean Square Difference: ', mse)

**日本語訳注**

librosaは本チュートリアルの冒頭でpip installしていますが、

Pythonの音楽分析用モジュール 「LibROSA」の意味です。

https://librosa.org/doc/latest/index.html

---


メルスペクトログラム
--------------

メルスケールスペクトログラムはスペクトログラムとメルスケール変換の組み合わせです。

``torchaudio``には``Spectrogram``と``MelScale``を組み合わせた``MelSpectrogram``変換が実装されています。



In [ ]:
waveform, sample_rate = get_speech_sample()

n_fft = 1024
win_length = None
hop_length = 512
n_mels = 128

mel_spectrogram = T.MelSpectrogram(
    sample_rate=sample_rate,
    n_fft=n_fft,
    win_length=win_length,
    hop_length=hop_length,
    center=True,
    pad_mode="reflect",
    power=2.0,
    norm='slaney',
    onesided=True,
    n_mels=n_mels,
)

melspec = mel_spectrogram(waveform)
plot_spectrogram(
    melspec[0], title="MelSpectrogram - torchaudio", ylabel='mel freq')

### librosaとの比較

比較として、``librosa``で同様のメルスケールスペクトログラムを生成します。

**注意** 

現状では、``htk=True``時のみ結果は一致します。

``htk=False``時の結果一致は``torchaudio``はサポートしていません。



In [ ]:
melspec_librosa = librosa.feature.melspectrogram(
    waveform.numpy()[0],
    sr=sample_rate,
    n_fft=n_fft,
    hop_length=hop_length,
    win_length=win_length,
    center=True,
    pad_mode="reflect",
    power=2.0,
    n_mels=n_mels,
    norm='slaney',
    htk=True,
)
plot_spectrogram(
    melspec_librosa, title="MelSpectrogram - librosa", ylabel='mel freq')

mse = torch.square(melspec - melspec_librosa).mean().item()
print('Mean Square Difference: ', mse)

MFCC (メル周波数ケプストラム係数)
----




In [ ]:
waveform, sample_rate = get_speech_sample()

n_fft = 2048
win_length = None
hop_length = 512
n_mels = 256
n_mfcc = 256

mfcc_transform = T.MFCC(
    sample_rate=sample_rate,
    n_mfcc=n_mfcc, melkwargs={'n_fft': n_fft, 'n_mels': n_mels, 'hop_length': hop_length})

mfcc = mfcc_transform(waveform)

plot_spectrogram(mfcc[0])

### librosaとの比較

In [ ]:
melspec = librosa.feature.melspectrogram(
  y=waveform.numpy()[0], sr=sample_rate, n_fft=n_fft,
  win_length=win_length, hop_length=hop_length,
  n_mels=n_mels, htk=True, norm=None)

mfcc_librosa = librosa.feature.mfcc(
  S=librosa.core.spectrum.power_to_db(melspec),
  n_mfcc=n_mfcc, dct_type=2, norm='ortho')

plot_spectrogram(mfcc_librosa)

mse = torch.square(mfcc - mfcc_librosa).mean().item()
print('平均二乗誤差: ', mse)

ピッチ
-----




In [ ]:
waveform, sample_rate = get_speech_sample()

pitch = F.detect_pitch_frequency(waveform, sample_rate)
plot_pitch(waveform, sample_rate, pitch)
play_audio(waveform, sample_rate)

カルディピッチ (beta)
------------------

カルディピッチ特徴量 [1] は自動音声認識アプリケーション用に調整されたピッチ検出手法です。

この機能はベータ版であり、``functional``形式でのみ利用可能です。


--- 

1. A pitch extraction algorithm tuned for automatic speech recognition

   Ghahremani, B. BabaAli, D. Povey, K. Riedhammer, J. Trmal and S.
   Khudanpur

   2014 IEEE International Conference on Acoustics, Speech and Signal
   Processing (ICASSP), Florence, 2014, pp. 2494-2498, doi:
   10.1109/ICASSP.2014.6854049. [[abstract](https://ieeexplore.ieee.org/document/6854049)], [[paper](https://danielpovey.com/files/2014_icassp_pitch.pdf)]



In [ ]:
waveform, sample_rate = get_speech_sample(resample=16000)

pitch_feature = F.compute_kaldi_pitch(waveform, sample_rate)
pitch, nfcc = pitch_feature[..., 0], pitch_feature[..., 1]

plot_kaldi_pitch(waveform, sample_rate, pitch, nfcc)
play_audio(waveform, sample_rate)

7.5 特徴量オーギュメンテーション（FEATURE AUGMENTATION）
====================




SpecAugment
-----------

[SpecAugment](https://ai.googleblog.com/2019/04/specaugment-new-data-augmentation.html)はスペクトログラムに対して適応可能な、よく用いられているオーギュメンテーション手法です。

``torchaudio``では``TimeStrech``、``TimeMasking``、
``FrequencyMasking``を用意しています。


### 時間方向の伸縮



In [ ]:
spec = get_spectrogram(power=None)
strech = T.TimeStretch()

rate = 1.2
spec_ = strech(spec, rate)
plot_spectrogram(spec_[0].abs(), title=f"Stretched x{rate}", aspect='equal', xmax=304)

plot_spectrogram(spec[0].abs(), title="Original", aspect='equal', xmax=304)

rate = 0.9
spec_ = strech(spec, rate)
plot_spectrogram(spec_[0].abs(), title=f"Stretched x{rate}", aspect='equal', xmax=304)

### 時間マスキング



In [ ]:
torch.random.manual_seed(4)

spec = get_spectrogram()
plot_spectrogram(spec[0], title="Original")

masking = T.TimeMasking(time_mask_param=80)
spec = masking(spec)

plot_spectrogram(spec[0], title="Masked along time axis")

### 周波数マスキング




In [ ]:
torch.random.manual_seed(4)

spec = get_spectrogram()
plot_spectrogram(spec[0], title="Original")

masking = T.FrequencyMasking(freq_mask_param=80)
spec = masking(spec)

plot_spectrogram(spec[0], title="Masked along frequency axis")

7.6 データセット（DATASETS）
========

``torchaudio``では、一般的によく使用されている音声データセットを簡単に使用することができます。

利用可能なデータセットの一覧は[公式ドキュメント](https://pytorch.org/audio/stable/datasets.html)を参照してください。

ここでは``YESNO``を例に使用方法を示します。


In [ ]:
YESNO_DOWNLOAD_PROCESS.join()

dataset = torchaudio.datasets.YESNO(YESNO_DATASET_PATH, download=True)

for i in [1, 3, 5]:
  waveform, sample_rate, label = dataset[i]
  plot_specgram(waveform, sample_rate, title=f"Sample {i}: {label}")
  play_audio(waveform, sample_rate)


# 日本語訳注
# YESNOデータセットでは、0か1のリスト（長さ8）に対応した音声を再生しています。
# 以下データセットから3つのデータを取り出します。
# データセットのラベルにはNo, Yesを示す長さ8のリストが入っています。
# 音声を再生して確かめてください。


【日本語訳注】

上記の音声を聞いていると、YesはCanのように聞こえるかと思います。

これは、今回のデータセットOpenSLRの仕様が

>「Yesno」
Sixty recordings of one individual saying yes or no in Hebrew; each recording is eight words long.

であり、Hebrew（ヘブライ語）での発音だからです。

<br>

Yesはヘブライ語で、כן　です。発音は以下のリンクの通りです（canに似ています）。

（発音）
https://ja.forvo.com/word/%D7%9B%D7%9F/

<br> 

（OpenSLR）
http://www.openslr.org/resources.php


以上。